# Deploy Nuclio functions with external python files

This notebook demonstrates how to deploy a nuclio function with mlrun working with external python file

In [ ]:
# mlrun server and client should be identical
!pip install nuclio-jupyter mlrun==0.5.3

### Setting configuration

In [2]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio

In [3]:
%nuclio cmd pip install textblob
%nuclio env TO_LANG=fr
%nuclio env CODE_CONTAINER=users
%nuclio env CODE_PATH=/nuclio/subdir
%nuclio env MOUNT_CODE_PATH=/code

%nuclio: setting 'TO_LANG' environment variable
%nuclio: setting 'CODE_CONTAINER' environment variable
%nuclio: setting 'CODE_PATH' environment variable
%nuclio: setting 'MOUNT_CODE_PATH' environment variable


Setting the path for the external function code for running localy from Jupyter or from a nuclio function 

In [4]:
import os
import sys

# The Python path will be determined by the run environment (local Jupyter or a Nuclio function)
hostname = os.environ["HOSTNAME"]
if 'jupyter' in hostname:
    
    # Local run in Jupyetr
    CODE_SYS_PATH = f'/v3io/{os.environ["CODE_CONTAINER"]}/{os.environ["V3IO_USERNAME"]}{os.environ["CODE_PATH"]}'
    
else:
    
    # Run in nuclio
    CODE_SYS_PATH = os.environ["MOUNT_CODE_PATH"]
        

In [15]:
CODE_SYS_PATH

'/v3io/users/edmond/subdir'

### NLP function example

This function is a simple nlp code that receive a string , translate it to another language and send the results back <br>
In the function there is an example of using an external function: val = ext.func()

In [14]:
sys.path.append(CODE_SYS_PATH)
import ext

from textblob import TextBlob

def handler(context, event):
    context.logger.info('This is an NLP example! ')

    # Get value from external function
    val = ext.func()
    context.logger.info_with("Value is ", value=str(val))
    
    # process and correct the text
    blob = TextBlob(str(event.body.decode('utf-8')))
    corrected = blob.correct()

    # debug print the text before and after correction
    context.logger.info_with("Corrected text", corrected=str(corrected), orig=str(blob))

    # calculate sentiments
    context.logger.info_with("Sentiment",
                             polarity=str(corrected.sentiment.polarity),
                             subjectivity=str(corrected.sentiment.subjectivity))

    # read target language from environment and return translated text
    lang = os.getenv('TO_LANG','fr')
    return str(corrected.translate(to=lang))

In [7]:
# nuclio: ignore
CODE_SYS_PATH

'/v3io/users/test/nuclio/subdir'

In [8]:
# nuclio: end-code

### Local Test

In [9]:
# nuclio: ignore
event = nuclio.Event(body=b'good morninng')
handler(context, event)

Python> 2020-11-17 09:48:51,917 [info] This is an NLP example! 
Python> 2020-11-17 09:48:51,918 [info] Value is : {'value': '5'}
Python> 2020-11-17 09:48:51,983 [info] Corrected text: {'corrected': 'good morning', 'orig': 'good morninng'}
Python> 2020-11-17 09:48:52,030 [info] Sentiment: {'polarity': '0.7', 'subjectivity': '0.6000000000000001'}


'Bonjour'

### Deploy with MLRun

In [10]:
from mlrun import code_to_function, mount_v3io

In [11]:
fn = code_to_function('nlp-example-new', handler='handler', kind='nuclio')

In order to access the external function we need to mount it to the path where the code resides

In [12]:
fn.apply(mount_v3io(remote=f'{os.environ["CODE_CONTAINER"]}/{os.environ["V3IO_USERNAME"]}{os.environ["CODE_PATH"]}', mount_path=os.environ['MOUNT_CODE_PATH']))

In [13]:
fn.spec.build.image

In [14]:
fn.deploy()

> 2020-11-17 09:49:04,286 [info] deploy started
[nuclio] 2020-11-17 09:49:06,415 (info) Build complete
[nuclio] 2020-11-17 09:49:13,533 done updating default-nlp-example-new, function address: 192.168.224.70:31312
> 2020-11-17 09:49:13,550 [warning] warning!, server (0.5.3-rc1) and client (0.5.3) ver dont match


'http://192.168.224.70:31312'

In [15]:
text = 'good morning'
fn.invoke('', text)

b'Bonjour'